In [2]:
from discopy import *
from discopy.function import *

ImportError: cannot import name 'Wire' from 'discopy.pregroup' (/Users/giovannidefelice/.pyenv/versions/3.7.4/lib/python3.7/site-packages/discopy/pregroup.py)

In [ ]:
def scalar_mult(scalar):
     return Function('scalar_mult({})'.format(repr(scalar)), Dim(1), Dim(1), lambda x: scalar * x)
    
def scalar_mults(dom, weights):
    result = Id(0)
    for i in range(dom):
        result = result @ scalar_mult(weights[i])
    return result
   
def bias(scalar):
    return Function('bias({})'.format(repr(scalar)), Dim(0), Dim(1), lambda x: np.array([scalar]))

def merge(cod, copies):
    @discofunc(cod * copies, cod, name='merge({}, {})'.format(cod, copies))
    def add(x):
        return np.array([np.sum([x[i + cod * j] for j in range(copies)]) for i in range(cod)])
    return add

def split(dom, copies):
    @discofunc(dom, dom * copies, name='split({}, {})'.format(dom, copies))
    def copy(x):
        return np.concatenate([x for i in range(copies)])
    return copy

@discofunc(1, 1)
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [6]:
def neuron(dom, cod, weights, beta=0): # weights is a 1d array of length dom, beta is a scalar bias
    return scalar_mults(dom, weights) @ bias(beta) >> merge(1, dom + 1) >> sigmoid >> split(1, cod)

In [12]:
neuron(4, 4, [0, 2.1, 0.3, 0.1])

scalar_mult(0) @ scalar_mult(2.1) @ scalar_mult(0.3) @ scalar_mult(0.1) @ bias(0) >> merge(1, 5) >> sigmoid >> split(1, 4)

In [18]:
from jax import jit, grad

print(jit(neuron(4, 4, [0, 2.1, 0.3, 0.1]))(np.array([0., 0.3, 1.2, 3.2])))
print(grad(lambda x: neuron(4, 1, [0, 2.1, 0.3, 0.1])(x)[0])(np.array([0., 0.3, 1.2, 3.2])))

[0.7875132 0.7875132 0.7875132 0.7875132]
[0.         0.35140604 0.05020086 0.01673362]


In [8]:
def layer(dom, cod, weights, biases): # weights is an array of size: cod x dom (note cod = number of neurons)
    neurons = Id(0)                   # biases is a 1d array of length cod
    for i in range(cod):
        neurons = neurons @ neuron(dom, 1, weights[i], biases[i])
    return split(dom, cod) >> neurons

In [9]:
layer(3, 2, [[0, 0, 0], [0, 0, 0]], [0, 0])(np.array([4, 1, 10]))

DeviceArray([0.5, 0.5], dtype=float32)